# Sanity check

In [1]:
!pwd

/home/amar/Desktop/projects/nfs_object_detection


In [2]:
! which python

/home/amar/Desktop/projects/nfs_object_detection/env/bin/python


# Imports

In [3]:
import os
import cv2
import numpy as np
import shutil
import random

# Paths and files

In [19]:
project_dir = os.getcwd()
paths = {
    'raw_video': os.path.join(os.getcwd(),'data','raw','videos'),
    'raw_image' : os.path.join(os.getcwd(),'data','raw','images'),
    'processed_video': os.path.join(os.getcwd(),'data','processed','videos'),
    'processed_image' : os.path.join(os.getcwd(),'data','processed','images'),
    'yolov3_tiny_config': os.path.join(project_dir, 'my_models', 'yolov3_tiny', 'yolov3_tiny.cfg'), 
    'yolov3_tiny_weights': os.path.join(project_dir, 'my_models', 'yolov3_tiny', 'yolov3_tiny.weights'),
    'yolov3_tiny_model': os.path.join(project_dir, 'my_models', 'yolov3_tiny', 'yolov3_tiny.h5'),
    'yolov3_320_config': os.path.join(project_dir, 'my_models', 'yolov3_320', 'yolov3_320.cfg'), 
    'yolov3_320_weights': os.path.join(project_dir, 'my_models', 'yolov3_320', 'yolov3_320.weights'),
    'yolov3_320_model': os.path.join(project_dir, 'my_models', 'yolov3_320', 'yolov3_320.h5'),
    'labels_dir': os.path.join(project_dir, 'data', 'labels'),
    'images_dir': os.path.join(project_dir, 'data', 'images'),
    'final_imgs_labels_dir': os.path.join(project_dir, 'colab', 'data', 'obj'),
    'colab_train.txt': os.path.join(project_dir, 'colab', 'data', 'train.txt'),
    'colab_test.txt': os.path.join(project_dir, 'colab', 'data', 'test.txt')
}

# Video to image

In [22]:
input_video_path = os.path.join(paths['raw_video'], 'video3.avi')
output_image_dir = os.path.join(paths['raw_image'])

In [23]:
#* Save the frames needed in the set wanted_frames
cap = cv2.VideoCapture(input_video_path)
count = 0
total_frames = cap.get(7)
# print(total_frames)
wanted_frames = set()
try:
   while True:
      ret, frame = cap.read()
      cv2.imshow('Input Video', frame)
      if cv2.waitKey(0) == ord('k'):
         wanted_frames.add(count)
      elif cv2.waitKey(0) == ord('q'):
         break
      count += 1
except:
   # print(count)
   print('Video not read, exiting..')

cv2.destroyAllWindows()
cap.release()
# print('wanted_frames:', wanted_frames)


Video not read, exiting..


In [33]:
print(len(wanted_frames))

8412


In [32]:
#* Save every 20th frame from the wanted_frames set
cap = cv2.VideoCapture(input_video_path)
count = 0
kept = 0
try:
    while True:
        ret, frame = cap.read()
        cv2.imshow('Input Video', frame)
        if count in wanted_frames:
            kept += 1
            if kept % 20 == 0:
                cv2.imwrite(os.path.join(output_image_dir, str(count) + '.jpg'), frame)
        if cv2.waitKey(10) == ord('q'):
            break
        count += 1
except Exception as e:
    cv2.destroyAllWindows()
    print(e)

# Colab Data preparation

In [8]:
paths

{'raw_video': '/home/amar/Desktop/projects/nfs_object_detection/data/raw/videos',
 'raw_image': '/home/amar/Desktop/projects/nfs_object_detection/data/raw/images',
 'processed_video': '/home/amar/Desktop/projects/nfs_object_detection/data/processed/videos',
 'processed_image': '/home/amar/Desktop/projects/nfs_object_detection/data/processed/images',
 'yolov3_tiny_config': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_tiny/yolov3_tiny.cfg',
 'yolov3_tiny_weights': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_tiny/yolov3_tiny.weights',
 'yolov3_tiny_model': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_tiny/yolov3_tiny.h5',
 'yolov3_320_config': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_320/yolov3_320.cfg',
 'yolov3_320_weights': '/home/amar/Desktop/projects/nfs_object_detection/my_models/yolov3_320/yolov3_320.weights',
 'yolov3_320_model': '/home/amar/Desktop/projects/nfs_object_detection/my_models/

In [11]:
# Storing the labels and images in a list to see which of the images were not labelled
labels = list()
images = list()

for dirpath, dirnames, filenames in os.walk(paths['labels_dir']):
    for file in filenames:
        labels.append(file[0:-4])

for dirpath, dirnames, filenames in os.walk(paths['images_dir']):
    for file in filenames:
        images.append(file[0:-4])

labels = set(labels)
images = set(images)

# Find the ones that are common
images_and_labels = labels.intersection(images)
len(images_and_labels)

335

## colab/data/obj folder

In [25]:
for i, image_id in enumerate(images_and_labels):
    img_src_path = os.path.join(paths['images_dir'], image_id + '.jpg')
    label_src_path = os.path.join(paths['labels_dir'], image_id + '.txt')
    dst_path = os.path.join(project_dir, 'colab', 'data', 'obj')
    shutil.copy(img_src_path, dst_path)
    shutil.copy(label_src_path, dst_path)
    print('{} files done'.format(i))

0 files done
1 files done
2 files done
3 files done
4 files done
5 files done
6 files done
7 files done
8 files done
9 files done
10 files done
11 files done
12 files done
13 files done
14 files done
15 files done
16 files done
17 files done
18 files done
19 files done
20 files done
21 files done
22 files done
23 files done
24 files done
25 files done
26 files done
27 files done
28 files done
29 files done
30 files done
31 files done
32 files done
33 files done
34 files done
35 files done
36 files done
37 files done
38 files done
39 files done
40 files done
41 files done
42 files done
43 files done
44 files done
45 files done
46 files done
47 files done
48 files done
49 files done
50 files done
51 files done
52 files done
53 files done
54 files done
55 files done
56 files done
57 files done
58 files done
59 files done
60 files done
61 files done
62 files done
63 files done
64 files done
65 files done
66 files done
67 files done
68 files done
69 files done
70 files done
71 files done
72

In [27]:
for dirpath, dirnames, filenames in os.walk(paths['final_imgs_labels_dir']):
    print(dirpath, len(dirnames), len(filenames))

/home/amar/Desktop/projects/nfs_object_detection/colab/data/obj 0 670


## Train and Test .txt files

In [14]:
len(images_and_labels)

335

In [18]:
test = set(random.sample(images_and_labels, 40))
train = images_and_labels - test
len(test), len(train), len(test)/(len(test) + len(train))

(40, 295, 0.11940298507462686)

In [22]:
with open(paths['colab_test.txt'], 'w') as f:
    for test_image in test:
        f.write(os.path.join('data/obj/' + test_image + '.jpg' + '\n'))

In [23]:
with open(paths['colab_train.txt'], 'w') as f:
    for train_image in train:
        f.write(os.path.join('data/obj/' + train_image + '.jpg' + '\n'))

## Models